## Spark Initialization

In [1]:
# Import findspark to read SPARK_HOME and HADOOP_HOME
import findspark
findspark.init()

In [2]:
# Import required library
from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [3]:
# Print Spark object ID
print(spark)

## Loading Data using Spark

In [4]:
# Datasets can be downloaded from https://www.kaggle.com/new-york-state/nys-511-ny-events

df = spark.read.csv("D:/TC/6BigData/Dataset/nys-511-ny-events/511-ny-events-beginning-2010.csv", header=True, inferSchema=True)

In [5]:
df.count()

1674327

In [6]:
df.show()

+--------------------+--------------------+--------------------+---------------+--------------------+-----------+-----+-------------------+-------------------+--------------------+--------------------------+---------+----------+
|          Event Type|   Organization Name|       Facility Name|      Direction|                City|     County|State|        Create Time|         Close Time|   Event Description|Responding Organization Id| Latitude| Longitude|
+--------------------+--------------------+--------------------+---------------+--------------------+-----------+-----+-------------------+-------------------+--------------------+--------------------------+---------+----------+
|        construction|MTA NYC Transit S...|              D Line|    Bronx bound|      from Manhattan|   New York|   NY|2016-06-30 22:58:00|2016-07-01 05:15:00|Due to Constructi...|      MTA NYC Transit S...|40.769205|-73.984887|
|        construction|Port Authority Ne...|George Washington...|      Westbound|    

In [7]:
df.schema

StructType(List(StructField(Event Type,StringType,true),StructField(Organization Name,StringType,true),StructField(Facility Name,StringType,true),StructField(Direction,StringType,true),StructField(City,StringType,true),StructField(County,StringType,true),StructField(State,StringType,true),StructField(Create Time,TimestampType,true),StructField(Close Time,TimestampType,true),StructField(Event Description,StringType,true),StructField(Responding Organization Id,StringType,true),StructField(Latitude,StringType,true),StructField(Longitude,StringType,true)))

In [8]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("ny_events")

In [9]:
result = spark.sql("SELECT DISTINCT `Event Type` FROM ny_events")

In [10]:
result.show()

+--------------------+
|          Event Type|
+--------------------+
|operational activ...|
|                 fog|
|accident, disable...|
|barrier repairs, ...|
|  Guard rail repairs|
|         broken rail|
|accident,accident...|
|Moveable Barrier ...|
|single line traff...|
|delays,stopped tr...|
|delays,traffic he...|
|objects falling f...|
|road reconstructi...|
|road maintenance ...|
|numerous accident...|
|CCTV repairs,test...|
|accident,capacity...|
|            lacrosse|
|moving operation,...|
|    show, exhibition|
+--------------------+
only showing top 20 rows



## Data Mining Process 

In [11]:
# Menampilkan data kejadian berdasarkan lokasi terjadinya

query1 = spark.sql("SELECT `Event Type`, `Event Description`, Latitude, Longitude, COUNT(`Event Type`) \
                    FROM ny_events \
                    GROUP BY `Event Type`, `Event Description`, Latitude, Longitude \
                    ORDER BY COUNT('Event Type') DESC")

In [12]:
query1.show()

+--------------------+--------------------+----------+------------+-----------------+
|          Event Type|   Event Description|  Latitude|   Longitude|count(Event Type)|
+--------------------+--------------------+----------+------------+-----------------+
|              delays|NYSDOT - Region 1...|     40.73|      -73.92|              766|
|     parking related|Port Authority NY...|   40.7692|    -73.8646|              763|
|              delays|NYSDOT - Region 1...|     40.73|      -73.91|              661|
|              delays|NYSDOT - Region 1...|     40.71|      -73.95|              612|
|              delays|NJ DOT - STMC: De...|     40.85|      -74.17|              606|
|       special event|MTA Bridges & Tun...|     40.74|      -73.96|              469|
|              delays|Delays on NJ 3 we...| 40.855328|  -74.178171|              467|
|       special event|MTA Bridges & Tun...|     40.69|      -74.01|              462|
|operational activity|Port Authority NY...|     40.85|

In [13]:
# Menampilkan nama organisasi yang paling sering menangani suatu kejadian

query2 = spark.sql("SELECT `Organization Name`, COUNT(`Organization Name`) \
                    FROM ny_events \
                    GROUP BY `Organization Name`\
                    ORDER BY COUNT(`Organization Name`) DESC")

In [14]:
query2.show()

+--------------------+------------------------+
|   Organization Name|count(Organization Name)|
+--------------------+------------------------+
|       NJ DOT - STMC|                  183320|
|  NYSDOT - Region 11|                  167607|
|              NYSDOT|                  141743|
|   NYSDOT - Region 8|                  137675|
|  NJ DOT - TOC South|                  122551|
|NYS Thruway Autho...|                   97991|
|   NJ Turnpike Auth.|                   74660|
|MTA NYC Transit S...|                   59854|
|CT DOT-Newington-...|                   51944|
|  NYSDOT - Region 10|                   51593|
|                 WTA|                   51473|
|New Jersey DOT - ...|                   45296|
|CT DOT-Bridgeport...|                   43407|
|New York State De...|                   40363|
|MTA Long Island R...|                   37043|
|Port Authority NY/NJ|                   35879|
| MTA NYC Transit Bus|                   30344|
|New Jersey Depart...|                  

In [15]:
# Menampilkan jumlah kejadian yang dimulai per harinya

query3 = spark.sql("SELECT TO_DATE(`Create Time`, 'MM/DD/YYYY') AS Date, COUNT(`Create Time`) \
                    FROM ny_events \
                    GROUP BY Date \
                    ORDER BY Date DESC")

In [16]:
query3.show()

+----------+------------------+
|      Date|count(Create Time)|
+----------+------------------+
|2016-06-30|               892|
|2016-06-29|               856|
|2016-06-28|               817|
|2016-06-27|               869|
|2016-06-26|               368|
|2016-06-25|               444|
|2016-06-24|               938|
|2016-06-23|               947|
|2016-06-22|               969|
|2016-06-21|               951|
|2016-06-20|               926|
|2016-06-19|               370|
|2016-06-18|               464|
|2016-06-17|               949|
|2016-06-16|              1014|
|2016-06-15|               978|
|2016-06-14|               973|
|2016-06-13|               814|
|2016-06-12|               365|
|2016-06-11|               386|
+----------+------------------+
only showing top 20 rows



In [17]:
# Menampilkan jumlah kejadian berdasarkan jenis kejadian dan rute tempat terjadinya

query4 = spark.sql("SELECT `Event Type`, `Facility Name`, Direction, COUNT(`Event Type`) \
                    FROM ny_events \
                    GROUP BY `Event Type`, `Facility Name`, Direction \
                    ORDER BY COUNT(`Event Type`) DESC")

In [18]:
query4.show()

+--------------------+--------------------+----------+-----------------+
|          Event Type|       Facility Name| Direction|count(Event Type)|
+--------------------+--------------------+----------+-----------------+
|              delays|               I-278| eastbound|             9195|
|            accident|  I-90 - NYS Thruway|      west|             8905|
|            accident|  I-90 - NYS Thruway|      east|             8787|
|            accident|  I-87 - NYS Thruway|     north|             8009|
|            accident|  I-87 - NYS Thruway|     south|             7889|
|              delays|               I-278| westbound|             7099|
|road maintenance ...|  I-90 - NYS Thruway|      east|             6841|
|              delays|                I95S|southbound|             6710|
|road maintenance ...|  I-90 - NYS Thruway|      west|             6146|
|    disabled vehicle|Taconic State Par...|northbound|             5970|
|              delays|                I95N|northbou

In [19]:
# Menampilkan daerah yang paling sering menjadi tempat kejadian

query5 = spark.sql("SELECT City, County, State, COUNT(City) \
                    FROM ny_events \
                    GROUP BY City, County, State \
                    ORDER BY COUNT(City) DESC")

In [20]:
query5.show()

+--------------+-----------+-----+-----------+
|          City|     County|State|count(City)|
+--------------+-----------+-----+-----------+
|      New York|     Queens|   NY|      48445|
|     Manhattan|   New York|   NY|      46367|
|      New York|      Bronx|   NY|      28691|
|      New York|      Kings|   NY|      27500|
|      New York|   New York|   NY|      26277|
|        Newark|      Essex|   NJ|      24817|
|      Brooklyn|      Kings|   NY|      15964|
|   Jersey City|     Hudson|   NJ|      14752|
|        Queens|     Queens|   NY|      12879|
|    Greenburgh|Westchester|   NY|      12357|
|      HARTFORD|   Hartford|   CT|      10641|
| Staten Island|   Richmond|   NY|      10036|
|Suffolk County|    Suffolk|   NY|       9118|
|     Hempstead|     Nassau|   NY|       9067|
|       Clifton|    Passaic|   NJ|       8672|
|       Yonkers|Westchester|   NY|       8280|
|      Bellmawr|     Camden|   NJ|       8182|
|  Hamilton Twp|     Mercer|   NJ|       7510|
|      Fort L